In [30]:
#2019.10.18 准备写一个大盘分析程序，将相关数据显示在网页上，然后后台定时运行，分析每天结果，我想看时打开网页就行。
#2019.10.24 实现往前算指定天数ndays，将index_dailybasic中的总市值与每日指数中成交量、成交额合并一个DF显示

In [53]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import tushare as ts
from datetime import date, datetime, timedelta
pro = ts.pro_api() 
#初始化tushare

In [65]:
#本段采集深证官网首页大盘信息，id=a,变量名前可以加a防重复。
opt = Options()
opt.add_argument('--headless')
# 创建chrome无界面对象
driver = webdriver.Chrome(options=opt)
#设置无界面
driver.get('http://www.szse.cn/')
#访问深证官网
time.sleep(3)
#停止等待，防止取出空数据
a_a=driver.find_elements_by_class_name('tab-pane')
#大盘情况放在class为tab-pane名字中，find_elements找出所有的tab_pane
a_c=[]
for a_b in a_a:
    if a_b.text!='':
        #一共能找到四个class，其中三个为空，要过滤掉
        a_c=a_b.text.split('\n') 
        #以\n分割字符串
table_name=a_c[::2]
#从0取值，步长2，取出列名
table_name.append('数据采集时间')
#增加列名：数据采集时间
table_value=a_c[1::2]
#从1取值，步长2，取出数值
a_d=driver.find_elements_by_class_name('tip')
#时间放在tip中，首页中一共有两个tip，取最后一个
a_f=[]
for a_e in a_d:
    a_f.append(a_e.text)
table_value.append(a_f[-1])
#对应增加采集到的日期
df1 = pd.DataFrame([table_value], columns=table_name) 
#df1深证官网的大盘数据

driver.quit()





In [67]:
#本段采集上证官网首页大盘信息，id=b,变量名前可以加a防重复。
import requests        #导入requests包
url = 'http://www.sse.com.cn/'
strhtml = requests.get(url)        #Get方式获取网页数据
strhtml.encoding=strhtml.apparent_encoding
print(strhtml.text)





<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>上海证券交易所</title>
<meta name="description" content="">
<meta name="author" content="上海证券交易所">
<meta name="renderer" content="webkit">
<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
<meta name="apple-mobile-web-app-capable" content="yes">
<meta name="mobile-web-app-capable" content="yes">
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=0">
<meta name="apple-mobile-web-app-title" content="上海证券交易所">
<meta property="wb:webmaster" content="6983a1253ca799d0" />
<script>var col_id=8301; var col_old_id="http://www.sse.com.cn/";</script>


<!-- CSS -->
<link rel="stylesheet" href="http://www.sse.com.cn/css/sse_indexcss.css;wa246a632c8f28065b">
<link rel="stylesheet" href="http://www.sse.com.cn/css/sse_indexcss_20181217.css;wa68cd584092d2af1d">
<!--[if IE 7]>
<link href="http://www.sse.com.cn/css/ie7fix.min.css;wa36f6f46246274f02" rel="stylesheet" type="text/css">
<![endif

In [57]:
driver.get('http://www.sse.com.cn/')

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=63364): Max retries exceeded with url: /session/d49a3d13a990d206b4462c21cdbfb140/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000168D3076438>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。'))

In [ ]:
dr

In [31]:
#2019.10.24 本段程序显示上证大盘情况。实现往前算指定天数ndays，将index_dailybasic中的总市值与每日指数中成交量、成交额合并一个DF显示
ndays=10
#ndays将来计划是由网页提交给数据库，可以由用户更改。
delta = timedelta(days=8)  
#timedelta可以实现日期时间直接相加，例子如下：
#now = datetime.now()
#day = date.today()
#n_days_after = now + delta  #当前日期推迟n天之后的时间
#n_days_forward  = now - delta  #当前日期向前推n天的时间
day=date.today()
#取今天的时间
end_date=(day-timedelta(days=1)).strftime('%Y%m%d')
#截止时间为前一天,#默认输出是2019-10-24的格式，用strftime换算成20191024
start_date=(day-timedelta(days=ndays)).strftime('%Y%m%d')
#开始时间根据ndays的不同，可以变化
dfa = pro.index_dailybasic(start_date=start_date,end_date=end_date,ts_code='000001.SH',fields='ts_code,trade_date,total_mv,turnover_rate')
#index_dailybasic提供大盘指数每日行情，目前主要是目前只提供上证综指，深证成指，上证50
#中证500，中小板指，创业板指的每日指标数据
df1 = pro.index_daily(ts_code='000001.SH',start_date=start_date,end_date=end_date,fields='ts_code,trade_date,close,change,pct_chg,vol,amount')
#指数每日行情，用主要输出参数close,open,high,low,pre_close
#change涨跌点相对昨天收价，pct_chg涨跌幅相对昨天收盘价
#vol成交量手，amount成交额千元
df1['total_mv']=dfa.total_mv
#将df1的total_mv合并df2
#dataframe增加一列用法，df['new']=[1,2,3,4] 
#dataframe增加一行用法，加loc,df.loc['new']=[1,2,3,4]
#--------------------我是分割线------------------------------------------------------------
#2019.10.19
#原本想将运行结果保存到数据库，然后用PHP读出来显示到网页上，今天突然想到PYTHON能不能直接保存结果为HTML？百度一下，居然发现DATAFRAME可以
#直接存为HTML文件，这下省事了。

df1

,ts_code,trade_date,close,change,pct_chg,vol,amount,total_mv
0,000001.SH,20191023,2941.6188,-12.7611,-0.4319,123812881.0,139653650.5,3.862913e+13
1,000001.SH,20191022,2954.3799,14.7620,0.5022,117482048.0,130381741.2,3.877605e+13
2,000001.SH,20191021,2939.6179,1.4766,0.0503,132475107.0,145685881.2,3.860444e+13
3,000001.SH,20191018,2938.1413,-39.1929,-1.3164,149990678.0,162018272.3,3.852931e+13
4,000001.SH,20191017,2977.3342,-1.3782,-0.0463,122527614.0,136051252.1,3.902802e+13
5,000001.SH,20191016,2978.7124,-12.3335,-0.4123,149885496.0,166791549.4,3.904096e+13
6,000001.SH,20191015,2991.0459,-16.8375,-0.5598,155376904.0,169531932.5,3.920118e+13
7,000001.SH,20191014,3007.8834,34.2276,1.1510,208614802.0,221201755.7,3.939485e+13


In [34]:
#2019.10.18 准备写一个大盘分析程序，将相关数据显示在网页上，然后后台定时运行，分析每天结果，我想看时打开网页就行。
#2019.10.24 本段程序显示深证大盘情况

dfa1 = pro.index_dailybasic(start_date=start_date,end_date=end_date,ts_code='399001.SZ',fields='ts_code,trade_date,total_mv,turnover_rate')
#index_dailybasic提供大盘指数每日行情，目前主要是目前只提供上证综指，深证成指，上证50
#中证500，中小板指，创业板指的每日指标数据
df2 = pro.index_daily(ts_code='399001.SZ',start_date=start_date,end_date=end_date,fields='ts_code,trade_date,close,change,pct_chg,vol,amount')
#指数每日行情，用主要输出参数close,open,high,low,pre_close
#change涨跌点相对昨天收价，pct_chg涨跌幅相对昨天收盘价
#vol成交量手，amount成交额千元
df2['total_mv']=dfa1.total_mv
#将df1的total_mv合并df2
#dataframe增加一列用法，df['new']=[1,2,3,4] 
#dataframe增加一行用法，加loc,df.loc['new']=[1,2,3,4]
#--------------------我是分割线------------------------------------------------------------
#2019.10.19
#原本想将运行结果保存到数据库，然后用PHP读出来显示到网页上，今天突然想到PYTHON能不能直接保存结果为HTML？百度一下，居然发现DATAFRAME可以
#直接存为HTML文件，这下省事了。
df2



,ts_code,trade_date,close,change,pct_chg,vol,amount,total_mv
0,399001.SZ,20191023,9567.7488,-74.3400,-0.7710,215975821.0,228668699.0,1.486832e+13
1,399001.SZ,20191022,9642.0888,88.5159,0.9265,207552391.0,226621129.8,1.499340e+13
2,399001.SZ,20191021,9553.5729,20.0677,0.2105,209913943.0,224397554.7,1.486568e+13
3,399001.SZ,20191018,9533.5052,-111.8835,-1.1600,238042710.0,253797032.8,1.483966e+13
4,399001.SZ,20191017,9645.3887,3.3292,0.0345,206596230.0,225158089.2,1.501314e+13
5,399001.SZ,20191016,9642.0595,-29.6693,-0.3068,233148157.0,256657323.7,1.499648e+13
6,399001.SZ,20191015,9671.7288,-114.9090,-1.1741,266349452.0,289616552.2,1.504899e+13
7,399001.SZ,20191014,9786.6378,120.0550,1.2420,285360084.0,313869068.4,1.521191e+13


In [ ]:
#多个DATAFRAME写入同一个html的例子
import os
HEADER ='''<html>
            <head>
                <meta charset="UTF-8">
            </head>
            <body>
            a1ddeeed'''
FOOTER = '''</body>
            </html>'''

 
with open(os.path.join(os.getcwd(), 'a.html'), 'w') as f:
    f.write(HEADER)
    f.write(FOOTER)


In [ ]:
df2.to_html('files.html',escape=False,index=False,sparsify=True,border=1,index_names=True,header=True)